#**Llama 2**

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

#**Step 1: Install All the Required Packages**

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install transformers
!pip install sentence_transformers
!pip install boto3

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 13.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 12.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 71.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 40.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 10.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-6cnn6avw/overlay/local/bin
    changing mode of /tmp/pip-build-env-6cnn6avw/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-6cnn6avw/overlay

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**Step 2: Import All the Required Libraries**

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
import scipy
import pandas as pd
import os
import boto3
from tqdm import tqdm
import pandas as pd
import csv
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Step 3: Download the Model**

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

#**Step 4: Loading the Model**

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=2048
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

#**Step 5: Text Generation Task**

In [ ]:
dataset_tg = pd.read_csv("/content/sample_data/cnn_test.csv")

In [ ]:
dataset_tg

,id,article,highlights
0,f001ec5c4704938247d27a44948eebb37ae98d01,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,230c522854991d053fe98a718b1defa077a8efef,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,4495ba8f3a340d97a9df1476f8a35502bcce1f69,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,c27cf1b136cc270023de959e7ab24638021bc43f,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...
...,...,...,...
11485,0ac776a4dc09ca97c136f4314fed4defb48a361a,Telecom watchdogs are to stop a rip-off that a...,Operators are charging up to 20p a minute - ev...
11486,fe89a6a2e28d173e5ad4c6d814c15b95aa969e3f,The chilling reenactment of how executions are...,Bali Nine ringleaders will face the firing squ...
11487,ded2f535cd6ab95d11b5f4ea29bbf2b2d3c55c50,It is a week which has seen him in deep water ...,Hardy was convicted of domestic abuse against ...
11488,30ec5f280eee772a73d181bfc8514defd8026434,"Despite the hype surrounding its first watch, ...",Apple sold more than 61 million iPhones in the...


In [ ]:
from transformers import GPT2Tokenizer

max_allowed_tokens = 500
dataset_tg["processed_input"]=-1

def count_tokens(sentence, tokenizer):
    tokens = tokenizer.encode(sentence, add_special_tokens=True)
    return len(tokens)

def process_input(sentence, max_tokens, tokenizer):
    cnt = count_tokens(sentence, tokenizer)
    if cnt <= max_tokens:
        return cnt
    else:
        return -1

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

for i, row in tqdm(dataset_tg.iloc[0:1500].iterrows()):

  id = dataset_tg['id'][i]
  article = dataset_tg['article'][i]

  input_sentence = article
  dataset_tg["processed_input"][i] = process_input(input_sentence, max_allowed_tokens, tokenizer)

df_new = dataset_tg[dataset_tg["processed_input"] != -1]

0it [00:00, ?it/s]<ipython-input-38-8703eb712fc2>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_tg["processed_input"][i] = process_input(input_sentence, max_allowed_tokens, tokenizer)
Token indices sequence length is longer than the specified maximum sequence length for this model (1161 > 1024). Running this sequence through the model will result in indexing errors
1500it [00:10, 146.17it/s]


In [ ]:
df_new

,id,article,highlights,processed_input
3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,233
4,c27cf1b136cc270023de959e7ab24638021bc43f,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,434
7,0d3c8c276d079c4c225f034c69aa024cdab7869d,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,452
8,6222f33c2c79b80be437335eeb3f488509e92cf5,(CNN)Filipinos are being warned to be on guard...,"Once a super typhoon, Maysak is now a tropical...",387
9,2bd8ada1de6a7b02f59430cc82045eb8d29cf033,"(CNN)For the first time in eight years, a TV l...","Bob Barker returned to host ""The Price Is Righ...",141
...,...,...,...,...
1487,944923fe4ddd51c12d7283601c87bd383c0e41bd,Carla Suarez Navarro advanced to the final of ...,Carla Suarez Navarro advances to final of the ...,438
1488,98c659ed98e4710ac601270c36ecc8f509d63045,There was no special treatment for Lewis Fergu...,Lewis Ferguson was mucking out the stables as ...,316
1492,31e674c555a4a77cad272288f12dd759bca175cd,Sussex head coach Mark Robinson has warned tha...,Matt Prior picked up injury playing for Englan...,288
1493,b472c77171f1ae38dc08eb0ac88effd21e718b43,World No 1 Mark Selby demolished Gary Wilson 1...,World No 1 Mark Selby defeated Gary Wilson 10-...,250


In [ ]:
df_new[0:500].to_csv('CNN-test-sampled.csv')

In [ ]:
df = pd.read_csv("/content/CNN-test-sampled.csv")

In [ ]:
df

,Unnamed: 0,id,article,highlights,processed_input
0,3,a38e72fed88684ec8d60dd5856282e999dc8c0ca,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,233
1,4,c27cf1b136cc270023de959e7ab24638021bc43f,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,434
2,7,0d3c8c276d079c4c225f034c69aa024cdab7869d,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,452
3,8,6222f33c2c79b80be437335eeb3f488509e92cf5,(CNN)Filipinos are being warned to be on guard...,"Once a super typhoon, Maysak is now a tropical...",387
4,9,2bd8ada1de6a7b02f59430cc82045eb8d29cf033,"(CNN)For the first time in eight years, a TV l...","Bob Barker returned to host ""The Price Is Righ...",141
...,...,...,...,...,...
495,1377,fb46940d9938f9602005bdc42ed14f53d403b1e0,A freak snowstorm was captured on video engulf...,The cyclone travelled through Siberia’s third ...,367
496,1378,539d909e3b49ee58471494bafe5b2c716c55675c,Dogs are usually maestros when it comes to gam...,Fritz the Golden Retriever from California has...,372
497,1382,49b1c006d82b83fa844fe1458ab49d467b2f78d2,Three people are fortunate to be alive after a...,The hot air balloon crashed into the tower dur...,409
498,1386,b9d0189bd220b95e233aae5fee16a1b18d4bd156,A mysterious Easter Bunny has delivered chocol...,Residents of Alderholt woke up yesterday to di...,254


#**Step 6A: Generating Response**

#Prompt 1 - Simple Colon

In [ ]:
file_path = f'/content/drive/MyDrive/CNN_Simple_Prompt1.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iloc[0:500].iterrows()):

  id = df['id'][i]
  article = df['article'][i]
  original_highlights = df['highlights'][i]

  #Simple Colon Prompt
  prompt_template=f'''Article: {article}.
  Summary:
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  summarized_text = input_text.split("Summary:")[1]

  org_article_embeddings = model.encode(article)
  summarized_embeddings = model.encode(summarized_text)

  similarity = scipy.spatial.distance.cdist([org_article_embeddings], [summarized_embeddings], "cosine")[0]
  similarity_score = 1 - similarity[0]

  row = {'id': id, 'Article': article, 'Original_Highlights': original_highlights, 'Translated_Highlights':summarized_text, 'Similarity_Score': similarity_score}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))


0it [00:00, ?it/s]<ipython-input-13-19b188b40000>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
1it [01:39, 99.68s/it]Llama.generate: prefix-match hit
<ipython-input-13-19b188b40000>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
2it [02:54, 85.25s/it]Llama.generate: prefix-match hit
<ipython-input-13-19b188b40000>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
3it [03:35, 64.90s/it]Llama.generate: prefix-match hit
<ipython-input-13-19b188b40000>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

In [ ]:
result_df.to_csv('CNN_Simple_Prompt1.csv', index=False)

#Prompt 2 - Master Translator

In [ ]:
file_path = f'/content/drive/MyDrive/CNN_Master_Prompt2.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iloc[0:500].iterrows()):

  id = df['id'][i]
  article = df['article'][i]
  original_highlights = df['highlights'][i]

  #Master Translator Prompt
  prompt_template=f'''An Article is provided: {article}.
  The masterful article summarizer flawlessly summarizes the article:
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  summarized_text = input_text.split("The masterful article summarizer flawlessly summarizes the article:")[1]

  org_article_embeddings = model.encode(article)
  summarized_embeddings = model.encode(summarized_text)

  similarity = scipy.spatial.distance.cdist([org_article_embeddings], [summarized_embeddings], "cosine")[0]
  similarity_score = 1 - similarity[0]

  row = {'id': id, 'Article': article, 'Original_Highlights': original_highlights, 'Translated_Highlights':summarized_text, 'Similarity_Score': similarity_score}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))


In [ ]:
result_df

In [ ]:
result_df.to_csv('CNN_Master_Prompt2.csv', index=False)

#Prompt 3 - Master Encouraged

In [ ]:
file_path = f'/content/drive/MyDrive/CNN_Master_Prompt3.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iloc[0:500].iterrows()):

  id = df['id'][i]
  article = df['article'][i]
  original_highlights = df['highlights'][i]

  #Master Enhanced Colon Prompt
  prompt_template=f'''An Article is provided: {article}.
  The extremely efficient masterful article summarizer flawlessly summarizes the article: '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  summarized_text = input_text.split("The extremely efficient masterful article summarizer flawlessly summarizes the article:")[1]

  org_article_embeddings = model.encode(article)
  summarized_embeddings = model.encode(summarized_text)

  similarity = scipy.spatial.distance.cdist([org_article_embeddings], [summarized_embeddings], "cosine")[0]
  similarity_score = 1 - similarity[0]

  row = {'id': id, 'Article': article, 'Original_Highlights': original_highlights, 'Translated_Highlights':summarized_text, 'Similarity_Score': similarity_score}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))


0it [00:00, ?it/s]Llama.generate: prefix-match hit


In [ ]:
result_df

In [ ]:
result_df.to_csv('CNN_Master_Prompt3.csv', index=False)

#Prompt 4 - Master Discouraged

In [ ]:
file_path = f'/content/drive/MyDrive/LLM/CNN_Master_Prompt4.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iloc[0:500].iterrows()):

  id = df['id'][i]
  article = df['article'][i]
  original_highlights = df['highlights'][i]

  #Master Enhanced Colon Prompt
  prompt_template=f'''An Article is provided: {article}.
  The incompetent masterful article summarizer summarizes the article: '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  summarized_text = input_text.split("The incompetent masterful article summarizer summarizes the article:")[1]

  org_article_embeddings = model.encode(article)
  summarized_embeddings = model.encode(summarized_text)

  similarity = scipy.spatial.distance.cdist([org_article_embeddings], [summarized_embeddings], "cosine")[0]
  similarity_score = 1 - similarity[0]

  row = {'id': id, 'Article': article, 'Original_Highlights': original_highlights, 'Translated_Highlights':summarized_text, 'Similarity_Score': similarity_score}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))

0it [00:00, ?it/s]<ipython-input-13-6179d1133e94>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
1it [02:59, 179.86s/it]Llama.generate: prefix-match hit
<ipython-input-13-6179d1133e94>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
2it [04:18, 120.10s/it]Llama.generate: prefix-match hit
<ipython-input-13-6179d1133e94>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
3it [05:45, 105.20s/it]Llama.generate: prefix-match hit
<ipython-input-13-6179d1133e94>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

In [ ]:
result_df

In [ ]:
result_df.to_csv('CNN_Master_Prompt4.csv', index=False)

#Prompt 5 - Few Shot Prompt (1-Shot Simple Colon)

In [ ]:
file_path = f'/content/drive/MyDrive/CNN_FewShot_1_Prompt5.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iloc[0:500].iterrows()):

  id = df['id'][i]
  article = df['article'][i]
  original_highlights = df['highlights'][i]

  #Few Shot Prompt
  prompt_template=f'''
  Article: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, said it was a move toward greater justice. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. "Indeed, today brings us closer to our shared goals of justice and peace." Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly," she said. Rights group Human Rights Watch welcomed the development. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court's treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "What's objectionable is the attempts to undermine international justice, not Palestine's decision to join a treaty to which over 100 countries around the world are members." In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it "strongly" disagreed with the court's decision. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. "We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace," it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as "Palestine." While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN's Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.
  Summary: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June . Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

  Article: {article}.
  Summary:
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  match_ee = re.search(r'(?:Summary:.*?){2}(.*)', input_text, re.DOTALL)

  if match_ee:
      summarized_text = match_ee.group(1).strip()
  else:
      print("Match not found. The current_index, original index, article, highlights are as follows: ", i, id, article, original_highlights)
      break

  org_article_embeddings = model.encode(article)
  summarized_embeddings = model.encode(summarized_text)

  similarity = scipy.spatial.distance.cdist([org_article_embeddings], [summarized_embeddings], "cosine")[0]
  similarity_score = 1 - similarity[0]

  row = {'id': id, 'Article': article, 'Original_Highlights': original_highlights, 'Translated_Highlights':summarized_text, 'Similarity_Score': similarity_score}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))


0it [00:00, ?it/s]<ipython-input-14-0dd68e904828>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
1it [01:35, 95.32s/it]Llama.generate: prefix-match hit
<ipython-input-14-0dd68e904828>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
2it [02:23, 67.58s/it]Llama.generate: prefix-match hit
<ipython-input-14-0dd68e904828>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
3it [03:46, 74.63s/it]Llama.generate: prefix-match hit
<ipython-input-14-0dd68e904828>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

In [ ]:
result_df

In [ ]:
result_df.to_csv('CNN_FewShot_1_Prompt5.csv', index=False)